<a href="https://colab.research.google.com/github/HE-Yangmei/Final_Prj/blob/master/local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

下面这一步就是连接Google drive

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
!git clone https://github.com/affinelayer/pix2pix-tensorflow.git

Cloning into 'pix2pix-tensorflow'...
remote: Enumerating objects: 261, done.
remote: Total 261 (delta 0), reused 0 (delta 0), pack-reused 261
Receiving objects: 100% (261/261), 13.33 MiB | 26.71 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [4]:
!ls

adc.json  drive  pix2pix-tensorflow  sample_data


In [0]:
## !python pix2pix-tensorflow/tools/download-dataset.py facades

downloading https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz
extracting
done


风景图片上色，真实地文件路径不能有空格
epoch = 200, dataset = 700, 200X700 = 140,000的数据量
batchsize未知
global_step = 是在训练过程中变化的，只是里面的一个循环

In [7]:
!python pix2pix-tensorflow/pix2pix.py \
  --mode train \
  --output_dir drive/pix2pix/photos/model \
  --max_epochs 200 \
  --input_dir drive/pix2pix/photos/train \
  --lab_colorization

aspect_ratio = 1.0
batch_size = 1
beta1 = 0.5
checkpoint = None
display_freq = 0
flip = True
gan_weight = 1.0
input_dir = drive/pix2pix/photos/train
l1_weight = 100.0
lab_colorization = True
lr = 0.0002
max_epochs = 200
max_steps = None
mode = train
ndf = 64
ngf = 64
output_dir = drive/pix2pix/photos/model
output_filetype = png
progress_freq = 50
save_freq = 5000
scale_size = 286
seed = 393113590
separable_conv = False
summary_freq = 100
trace_freq = 0
which_direction = AtoB
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(

上面这个训练过程会自动保存模型

In [0]:
## saver = tf.train.Saver()
## saver.save(sess, './pix2pix_', global_step=5000)

查看训练细节，可是这个查看时间是与训练时间一样长

In [0]:
!tensorboard --logdir=drive/pix2pix/photos/model

测试的时候会自动写入一个html

In [0]:
!python pix2pix-tensorflow/pix2pix.py 
  --mode test \
  --output_dir drive/pix2pix/photos/test \
  --input_dir drive/pix2pix/photos/test \
  --checkpoint drive/pix2pix/photos/model

In [0]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'pix2pix-tensorflow/facades_train'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))